In [29]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [31]:
dfCrime = pd.read_csv('/Users/austincoffelt/Downloads/Crime_Data_from_2020_to_Present.csv')
print(dfCrime.shape)
dfCrime.head()

(1005198, 28)


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,200412582,09/09/2020 12:00:00 AM,09/09/2020 12:00:00 AM,630,4,Hollenbeck,413,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E AVENUE 28,NaN,34.0820,-118.2130


In [32]:
dfCrime['DATE OCC'] = pd.to_datetime(dfCrime['DATE OCC'])
dfCrime = dfCrime[(dfCrime['DATE OCC'] >= '2021-01-01') & (dfCrime['DATE OCC'] < '2022-01-01')]
print(dfCrime.shape)
dfCrime.head()

(209875, 28)


/var/folders/6z/0pv81j2524v8c80qsmnrbmcm0000gn/T/ipykernel_60899/1281132472.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dfCrime['DATE OCC'] = pd.to_datetime(dfCrime['DATE OCC'])


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
125803,210514632,10/08/2021 12:00:00 AM,2021-10-07,2200,5,Harbor,587,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,700 W 30TH ST,NaN,33.7170,-118.2917
126027,210309534,04/29/2021 12:00:00 AM,2021-04-28,843,3,Southwest,332,1,648,ARSON,...,IC,Invest Cont,648.0,NaN,NaN,NaN,2800 HILLCREST DR,NaN,34.0294,-118.3429
126195,211909242,05/30/2021 12:00:00 AM,2021-05-30,100,19,Mission,1963,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,9800 WOODMAN AV,NaN,34.2454,-118.4505
126891,210314198,08/10/2021 12:00:00 AM,2021-08-10,703,3,Southwest,376,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,WESTERN,39TH,34.0146,-118.3089
127806,210108171,03/30/2021 12:00:00 AM,2021-03-29,2100,1,Central,124,2,888,TRESPASSING,...,AO,Adult Other,888.0,NaN,NaN,NaN,2ND,LOS ANGELES,34.0510,-118.2445


In [33]:
violentCrimeCodes = [110, 113, 121, 122, 210, 220, 230, 231]
dfCrime = dfCrime[dfCrime['Crm Cd'].isin(violentCrimeCodes)]
print(dfCrime.shape)
dfCrime.head()

(22897, 28)


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
199865,211218123,08/21/2021 12:00:00 AM,2021-08-21,2351,12,77th Street,1215,1,210,ROBBERY,...,AO,Adult Other,210.0,NaN,NaN,NaN,VERMONT,58TH ST,33.9897,-118.2915
199878,210122790,12/25/2021 12:00:00 AM,2021-12-25,1900,1,Central,157,1,210,ROBBERY,...,IC,Invest Cont,210.0,NaN,NaN,NaN,5TH,TOWNE,34.0430,-118.2420
199879,210611673,06/24/2021 12:00:00 AM,2021-06-24,235,6,Hollywood,646,1,210,ROBBERY,...,IC,Invest Cont,210.0,NaN,NaN,NaN,IVAR,SELMA,34.0998,-118.3288
199881,210413995,10/17/2021 12:00:00 AM,2021-10-17,1030,4,Hollenbeck,411,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,IC,Invest Cont,230.0,NaN,NaN,NaN,AVENUE 19,HUMBOLDT ST,34.0761,-118.2238
199883,211911076,07/19/2021 12:00:00 AM,2021-07-19,730,19,Mission,1983,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,IC,Invest Cont,230.0,NaN,NaN,NaN,8400 N SEPULVEDA BL,NaN,34.2233,-118.4677


In [34]:
dfCrime = dfCrime[['Crm Cd', 'LON', 'LAT']]
dfCrime.head()

,Crm Cd,LON,LAT
199865,210,-118.2915,33.9897
199878,210,-118.2420,34.0430
199879,210,-118.3288,34.0998
199881,230,-118.2238,34.0761
199883,230,-118.4677,34.2233


In [36]:
# make our lon and lat into a point type for spacial join
dfCrime['geometry'] = dfCrime.apply(lambda row: Point(row['LON'], row['LAT']), axis=1)
dfCrime = dfCrime[['Crm Cd', 'geometry']]
print(dfCrime.shape)
dfCrime.head()

(22897, 2)


,Crm Cd,geometry
199865,210,POINT (-118.2915 33.9897)
199878,210,POINT (-118.242 34.043)
199879,210,POINT (-118.3288 34.0998)
199881,230,POINT (-118.2238 34.0761)
199883,230,POINT (-118.4677 34.2233)


In [41]:
# Read shapefile (automatically loads .shp, .shx, .dbf)
gdf = gpd.read_file("cb_2022_us_bg_500k/cb_2022_us_bg_500k.shp")

# View the first 5 rows
print(gdf.shape)
gdf.head()

(242297, 12)


,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER,geometry
0,04,019,940800,2,1500000US040199408002,040199408002,2,Block Group 2,BG,2033911373,68536,"POLYGON ((-112.41 31.857, -112.38 31.859, -112..."
1,04,003,000901,4,1500000US040030009014,040030009014,4,Block Group 4,BG,486316,0,"POLYGON ((-109.55 31.334, -109.55 31.339, -109..."
2,06,073,018102,3,1500000US060730181023,060730181023,3,Block Group 3,BG,324091,3579,"POLYGON ((-117.37 33.182, -117.37 33.182, -117..."
3,06,073,007200,3,1500000US060730072003,060730072003,3,Block Group 3,BG,720442,709604,"POLYGON ((-117.26 32.725, -117.26 32.726, -117..."
4,06,073,017702,3,1500000US060730177023,060730177023,3,Block Group 3,BG,317237,0,"POLYGON ((-117.3 33.052, -117.3 33.053, -117.3..."


In [42]:
# make sure points match the crs of gdf
gdfPoints = gpd.GeoDataFrame(
    dfCrime, 
    geometry='geometry',
    crs="EPSG:4269"  # WGS84 coordinate reference system
)
gdfPoints.head()

,Crm Cd,geometry
199865,210,POINT (-118.29 33.99)
199878,210,POINT (-118.24 34.043)
199879,210,POINT (-118.33 34.1)
199881,230,POINT (-118.22 34.076)
199883,230,POINT (-118.47 34.223)


In [43]:
# Basic spatial join (points within polygons)
result = gpd.sjoin(
    gdfPoints, 
    gdf,
    how="left",       # 'left', 'right', or 'inner'
    predicate="within" # Spatial relationship
)
print(result.shape)
result.head()

(22897, 14)


,Crm Cd,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER
199865,210,POINT (-118.29 33.99),8116.0,06,037,232602,1,1500000US060372326021,060372326021,1,Block Group 1,BG,266304.0,0.0
199878,210,POINT (-118.24 34.043),71087.0,06,037,206303,1,1500000US060372063031,060372063031,1,Block Group 1,BG,434791.0,0.0
199879,210,POINT (-118.33 34.1),14581.0,06,037,190701,1,1500000US060371907011,060371907011,1,Block Group 1,BG,214583.0,0.0
199881,230,POINT (-118.22 34.076),52934.0,06,037,199002,2,1500000US060371990022,060371990022,2,Block Group 2,BG,1035192.0,62385.0
199883,230,POINT (-118.47 34.223),214543.0,06,037,117530,3,1500000US060371175303,060371175303,3,Block Group 3,BG,252400.0,6267.0


In [44]:
crime = result['AFFGEOID'].value_counts()
crime

AFFGEOID
1500000US060372077111    160
1500000US060372063011    155
1500000US060372063031    148
1500000US060372088011    114
1500000US060372260021    113
                        ... 
1500000US060371897041      1
1500000US060371031021      1
1500000US060372976012      1
1500000US060371397051      1
1500000US060372640001      1
Name: count, Length: 2393, dtype: int64

In [49]:
# export as a csv
crime.to_csv('LA_Violent_Crime_BG.csv')